# Regresní metriky

V úvodním příkladu s krajinou jsme si metriku vymýšleli sami. Existuje 
samozřejmě řada standardním metrik, které se k hodnocení modelů používají.
Uvedeme si přehled nejnámějších z nich:

($y = y_1, \ldots, y_N$ jsou reálné hodnoty odezvy,
$\hat{y} = \hat{y}_1, \ldots, \hat{y}_N$ jsou predikované hodnoty)

-  **MSE (Mean squared error)** - průměr sumy čterců odchylek požadovaných výstupů od predikovaných výstupů 

    $$
    MSE(y, \hat{y}) = \frac{1}{N} \sum_{i=1}^N (y_i - \hat{y}_i)^2 
    $$

- **MAE (Mean absolute error)** - průměr absolutních hodnot odchylek požadovaných výstupů od predikovaných výstupů 

    $$
    MAE(y, \hat{y}) = \frac{1}{N} \sum_{i=1}^N |y_i - \hat{y}_i| 
    $$

- **$R^2$ score (Koeficient determinace)** - koeficient determinace vyjadřuje, jaký podíl variability 
 závislé proměnné (odezvy) model vyjadřuje. Dá se interpretovat tak, že říká, jak moc je náš model lepší než
 konstantní *baseline* daná jako průměr. 
$$
 R^2(y, \hat{y}) = 1 - \frac{MSE(model)}{MSE(baseline)} = 1 - \frac{MSE(y, \hat{y})}{MSE(y, \bar{y})}; 
 \qquad \mbox{kde}\quad \bar{y} = \frac{1}{N} \sum_{i=1}^N y_i
$$
$R^2$ skóre dosahuje maximálně hodnoty jedna, což znamená dokonalou predikci. 

Pokud nemáš ráda vzorečky, nelam si s nimi hlavu. Podívej se na následující příklad:


Vytvoříme dataframe, který bude obsahovat odezvy (sloupec "správně") a predikované hodnoty (sloupec "predikováno"). Pro jednoduchost hodnoty odezvy vygenerujeme náhodně a jejich "predikce" vytvoříme tak, že k těmto 
odezvám přičteme náhodné číslo z intervalu (-10, 10). 

In [89]:
import pandas as pd 
import random

df = pd.DataFrame({"správně": [random.randint(10, 90) for _ in range(10)]})
df["predikováno"] = df["správně"].apply(lambda predikovano: predikovano + random.randint(-10, 10))
df

,správně,predikováno
0,26,20
1,68,65
2,28,31
3,20,17
4,14,16
5,40,34
6,44,47
7,80,79
8,75,81
9,78,68


Spočteme si MSE, MAE a $R^2$ skóre. Napišme si na to funkci:

In [90]:
def spocti_metriky(skutecna_odezva, predikovano):
    chyby = pd.DataFrame()
    chyby["absolutní_chyba"] = abs(skutecna_odezva - predikovano)
    chyby["chyba_na_druhou"] = (skutecna_odezva - predikovano)**2
    baseline = skutecna_odezva.mean()
    chyby["chyba_na_druhou_baseline"] = (skutecna_odezva - baseline)**2

    MSE = chyby["chyba_na_druhou"].mean()
    MAE = chyby["absolutní_chyba"].mean()
    R2 = 1 - MSE/chyby["chyba_na_druhou_baseline"].mean()
    
    return chyby, MSE, MAE, R2

In [91]:
df_chyby, MSE, MAE, R2 = spocti_metriky(df["správně"], df["predikováno"])
# zobrazme si tabulku spolu s odpovídajícími chybami  
pd.concat([df, df_chyby], axis="columns")

,správně,predikováno,absolutní_chyba,chyba_na_druhou,chyba_na_druhou_baseline
0,26,20,6,36,453.69
1,68,65,3,9,428.49
2,28,31,3,9,372.49
3,20,17,3,9,745.29
4,14,16,2,4,1108.89
5,40,34,6,36,53.29
6,44,47,3,9,10.89
7,80,79,1,1,1069.29
8,75,81,6,36,767.29
9,78,68,10,100,942.49


In [92]:
# vypišme si hodnoty metrik
print(f"MSE = {MSE}")
print(f"MAE = {MAE}")
print(f"R2 = {R2}")

MSE = 24.9
MAE = 4.3
R2 = 0.9581660254364006


Z tabulky výše i ze zobrazených chyb vidíme, že MSE daleko více penalizuje *větší* chyby.

Zkusme si ještě nasimulovat řešení, které bude poměrně přesné (náhodná odchylka, kterou přičítáme, bude z intervalu (-1,1)).

In [93]:
df2 = pd.DataFrame()
df2["správně"] = df["správně"]
df2["predikováno"] = df2["správně"].apply(lambda predikovano: predikovano + random.uniform(-1., 1.))
df2

,správně,predikováno
0,26,25.542426
1,68,67.804820
2,28,28.443652
3,20,20.915919
4,14,14.991681
5,40,40.941535
6,44,44.878549
7,80,80.801804
8,75,75.156222
9,78,78.802822


Opět zorbrazme chyby:

In [94]:
df2_chyby, MSE_2, MAE_2, R2_2 = spocti_metriky(df2["správně"], df2["predikováno"])
pd.concat([df2, df2_chyby], axis="columns")

,správně,predikováno,absolutní_chyba,chyba_na_druhou,chyba_na_druhou_baseline
0,26,25.542426,0.457574,0.209374,453.69
1,68,67.804820,0.195180,0.038095,428.49
2,28,28.443652,0.443652,0.196827,372.49
3,20,20.915919,0.915919,0.838907,745.29
4,14,14.991681,0.991681,0.983431,1108.89
5,40,40.941535,0.941535,0.886489,53.29
6,44,44.878549,0.878549,0.771848,10.89
7,80,80.801804,0.801804,0.642890,1069.29
8,75,75.156222,0.156222,0.024405,767.29
9,78,78.802822,0.802822,0.644523,942.49


A vypišme si hodnoty metrik:

In [95]:
print(f"MSE = {MSE_2:.3f} (minule: {MSE})")   # :.2 znamená na dvě desetinná místa
print(f"MAE = {MAE_2:.3f} (minule: {MAE})")
print(f"R2  = {R2_2:.3f} (minule: {R2})")

MSE = 0.524 (minule: 24.9)
MAE = 0.658 (minule: 4.3)
R2  = 0.999 (minule: 0.9581660254364006)


Vidíme, že hodnoty MSE a MAE jsou teď výrazně menší než minule. To je proto, že "predikované" hodnoty jsou velmi blízko hodnotám skutečným. $R^2$ skóre vyšlo naopak vyšší (blíže jedné). 

Pozn.: Pozor, nyní vyšla  hodnota MAE vyšší než MSE, to je proto, že jednotlivé chyby jsou teď menší než jedna (mocnina je menší než absolutní hodnota). 